# Taking a look at the data

In [2]:
import pandas as pd
import numpy as np

In [3]:
#have these from API and web scraping
local_docs = 78 #local is web scraping
colab_docs = 8  #colab is api

### Put all gathered data into one dataframe

In [4]:
df = pd.DataFrame(columns =
                  ['index', 'Quest ID', 'Quest Title',
                   'Quest Description', 'Quest Summary', 'Quest Hand-in'])
#Create dataframe

In [5]:
quest_id_arr = []
quest_title_arr = []
quest_summary_arr = []
quest_description_arr = []
quest_handin_arr = []
for index in range(1, colab_docs+1):    #get the colab files
    df_temp = pd.read_csv(f'colab_dict_list_{index}.csv')
    df_temp.drop(df_temp.columns[0], axis = 1, inplace = True)
    quest_id_arr.append(df_temp.columns)
    quest_title_arr.append(df_temp.iloc[0].values.tolist())
    quest_description_arr.append(df_temp.iloc[1].values.tolist())
quest_title_arr = [item for list in quest_title_arr for item in list]
quest_description_arr = [item for list in quest_description_arr for item in list]
quest_id_arr = [item for list in quest_id_arr for item in list]
quest_id_arr =[int(i) for i in quest_id_arr]
for index in range(1, local_docs+1):
    df_temp = pd.read_csv(f'quest_dict_list_{index}.csv')
    df_temp.drop(df_temp.columns[0], axis = 1, inplace = True)
    quest_summary_arr.append(df_temp.iloc[0].values.tolist())
    quest_handin_arr.append(df_temp.iloc[1].values.tolist())
quest_summary_arr = [item for list in quest_summary_arr for item in list]
quest_handin_arr = [item for list in quest_handin_arr for item in list]
quest_summary_arr = [quest_summary_arr[i-1] for i in quest_id_arr]
quest_handin_arr = [quest_handin_arr[i-1] for i in quest_id_arr]


In [6]:
len(quest_id_arr)

27793

In [5]:
quest_id_arr[-5:]

[77429, 77430, 77432, 77437, 77679]

In [6]:
df['Quest ID'] = quest_id_arr
df['Quest Title'] = quest_title_arr
df['Quest Description'] = quest_description_arr
df['Quest Summary'] = quest_summary_arr
df['Quest Hand-in'] = quest_handin_arr
df['index'] = range(0, len(df))
df.set_index('index', inplace = True)

In [240]:
df.head()

,Quest ID,Quest Title,Quest Description,Quest Summary,Quest Hand-in
index,,,,,
0,2,Sharptalon's Claw,The mighty hippogryph Sharptalon has been slai...,"What have you there, <class>? Could it be....?",You have slain the beast?I owe you a great deb...
1,11,Riverpaw Gnoll Bounty,"Gnolls, brutish creatures with no decent busin...","Hail, <name>. Have you been killing Gnolls...?","I see you've been busy! You have our thanks, ..."
2,16,Give Gerard a Drink,"Farming is thirsty work, and I'm always lookin...",NaN,"Thanks, <name>! And come back if you want to ..."
3,23,Ursangous' Paw,The mighty bear Ursangous is no more!\n\nThe s...,"What is it, <race>?",The paw of Ursangous!Have you any idea what tr...
4,24,Shadumbra's Head,The once swift Shadumbra now lies still as you...,"Yes, <class>?",Shadumbra's Head?None can say how many of our ...


In [241]:
#Creating the dataframe
df.to_csv('Final Data.csv')

### Some data exploration

In [175]:
len(df)

27793

In [176]:
df.isna().sum()

Quest ID                 0
Quest Title              0
Quest Description     2251
Quest Summary        21019
Quest Hand-in         4292
dtype: int64

In [177]:
df.dtypes

Quest ID              int64
Quest Title          object
Quest Description    object
Quest Summary        object
Quest Hand-in        object
dtype: object

In [1]:
for j in df.columns:
    print((set([type(i) for i in df[str(j)]])))
#have some non-string entries, presumably all NaN type for missing data

NameError: name 'df' is not defined

##### Run checks to ensure all non-string entries are NaN

In [8]:
float_desc = []
for i in range (0, len(df)):
    if type(df['Quest Description'].iloc[i]) != str:
        if np.isnan(df['Quest Description'].iloc[i]):
            continue
        float_desc.append(i)

In [180]:
df.loc[float_desc]

,Quest ID,Quest Title,Quest Description,Quest Summary,Quest Hand-in
index,,,,,


In [9]:
float_summ = []
for i in range (0, len(df)):
    if type(df['Quest Summary'].iloc[i]) != str:
        if np.isnan(df['Quest Summary'].iloc[i]):
            continue
        float_summ.append(i)

In [182]:
df.loc[float_summ]

,Quest ID,Quest Title,Quest Description,Quest Summary,Quest Hand-in
index,,,,,


In [10]:
float_handin = []
for i in range (0, len(df)):
    if type(df['Quest Hand-in'].iloc[i]) != str:
        if np.isnan(df['Quest Hand-in'].iloc[i]):
            continue
        float_handin.append(i)

In [184]:
df.loc[float_handin]

,Quest ID,Quest Title,Quest Description,Quest Summary,Quest Hand-in
index,,,,,


In [11]:
#look at total text lengths
title_chars = 0
desc_chars = 0
summ_chars = 0
handin_chars = 0
for i in range (0, len(df)):
    try:
        title_chars += len(df['Quest Title'].iloc[i])
    except:
        pass
    try:
        desc_chars += len(df['Quest Description'].iloc[i])
    except:
        pass
    try:
        summ_chars += len(df['Quest Summary'].iloc[i])
    except:
        pass
    try:
        handin_chars += len(df['Quest Hand-in'].iloc[i])
    except:
        pass
print(f'Total characters for each text:\nTitles: {title_chars}\nDescriptions: '
      f'{desc_chars}\nSummaries: {summ_chars}\nHand-ins: {handin_chars}')

Total characters for each text:
Titles: 552283
Descriptions: 8444675
Summaries: 389169
Hand-ins: 3095644


In [13]:
#total length of possible input for model
title_chars+desc_chars

8996958

In [17]:
#actual input is less because of empty descriptions
i = 0
for j in range(0, len(df)):
    if type(df['Quest Description'].iloc[j]) == str:
        i+=1
i

25542

In [12]:
full_text = ''
for i in range (0, len(df)):
    for col in df.columns[1:]:
        try:
            full_text = full_text+str(df[col].iloc[i])
        except:
            pass
len(full_text)

12564457

In [187]:
print(sorted(set(full_text)))
print(len(set(full_text)))

['\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', 'ä', '�']
96


In [188]:
max_desc = 0
index = 0
for i in range(0, len(df)):
    try:
        L = len(df['Quest Description'].iloc[i])
        if L > max_desc:
            max_desc = L
            index = i
    except:
        pass
print(df['Quest Description'].iloc[index])
print(max_desc)

{name}, a proud {race} {class}. I know your name and your deeds. I can taste your regrets and your shame.

More importantly, I know what you might become. You can be the beacon for the Shadowlands to rally behind. The symbol my people need to set aside these petty squabbles and help fix the problems that afflict all reality.

Before you stands the Court of Harvesters, my most loyal and trusted venthyr. Where once there were seven, only four remain. 

Introduce yourself to them, for where they go, my power follows
524
